# Notebook for Calculating 

Compute seasonal cycle of precipitation over selected area


# Running the notebook
Execute the cells 1 by 1 by taping `ctrl + enter` or `shift + enter`

__Make sure to upload the file before continuing the rest of the notebook__

If you want to change the longitude and latitude bounds look in the data extraction section.

# Imports

In [ ]:
# File upload widegt
from ipywidgets import FileUpload
from io import BytesIO

# Data manipulation and storage
import xarray as xr

# Holoviews graphing
import hvplot.xarray
import holoviews as hv

In [ ]:
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs

gv.extension('bokeh', 'matplotlib')

# Data Upload

In [ ]:
# Create file uploader
file_upload = FileUpload()
file_upload

In [ ]:
bytes_data = BytesIO(file_upload.data[0])
ds = xr.open_dataset(bytes_data)

In [ ]:
ds[['precip', 'phis']].phis.hvplot()

In [ ]:
ensemble = gv.Dataset(ds).to(gv.Image, ['lon', 'lat'], 'precip')
ensemble.opts(projection=crs.Robinson())

# Data extraction 

Here we take only the variables we need from the dataset

In [ ]:
ds_precip = ds[['precip', 'phis']]

In [ ]:
# Select the desired zone
# /!\ Slice works based on the ordering in coordinates, here lat goes from +90 -> -90 and lon from -180 -> +180
ds_precip_zone = ds_precip.sel(lat=slice(35, 0), lon=slice(65, 85))
# Calculate the altitude from the phi parameter
ds_precip_zone['alti'] = ds_precip_zone['phis'] / 9.81

# Analysis

In [ ]:
ds_precip_zone.precip.hvplot.image(label='precip')

In [ ]:
ds_precip_zone.alti.hvplot.image(label='alti')

In [ ]:
# Extract values above a threshold
threshold = 1000
upper = ds_precip_zone.where(ds_precip_zone.alti >= threshold)

# Extract values below threshold but above sea level
lower = ds_precip_zone.where((ds_precip_zone.alti > 0) & (ds_precip_zone.alti < threshold))

In [ ]:
upper.hvplot.image()

In [ ]:
lower.hvplot.image()

# Calculate the mean value for each time step

In [ ]:
upper.precip.mean(dim=['lat', 'lon']).plot.line();

In [ ]:
lower.precip.mean(dim=['lat', 'lon']).plot.line();

# Calculate the sum over all locations at each time step

In [ ]:
lower.precip.sum(dim=['lat', 'lon']).plot.line();

In [ ]:
lower.precip.sum(dim=['lat', 'lon']).plot() + upper.precip.sum(dim=['lat', 'lon']).plot();

# Calculate the cumlative sum for each time step

In [ ]:
upper.precip.sum(dim=['lat', 'lon']).cumsum().plot.line();

In [ ]:
lower.precip.sum(dim=['lat', 'lon']).cumsum().plot.line();